In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q albumentations opencv-python-headless pillow tqdm pyyaml


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:0000:01


In [5]:
import cv2
import numpy as np
import albumentations as A
from pathlib import Path
from tqdm import tqdm
import shutil
import yaml
import json
from ultralytics import YOLO

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [6]:
def get_augmentation_pipeline():
    """Augmentation pipeline for drone landing conditions"""
    return A.Compose([
        # Lighting variations
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=1),
            A.RandomGamma(gamma_limit=(50, 150), p=1),
            A.CLAHE(p=1),
        ], p=0.7),
        # Weather effects
        A.OneOf([
            A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=1),
            A.RandomRain(slant_lower=-10, slant_upper=10, p=1),
            A.RandomSunFlare(src_radius=100, p=1),
        ], p=0.3),
        # Motion blur
        A.OneOf([
            A.MotionBlur(blur_limit=15, p=1),
            A.GaussianBlur(blur_limit=7, p=1),
            A.Defocus(radius=(3, 7), alias_blur=(0.1, 0.3), p=1),
        ], p=0.4),
        # Noise
        A.OneOf([
            A.GaussNoise(var_limit=(10, 50), p=1),
            A.ISONoise(p=1),
            A.CoarseDropout(max_holes=5, max_height=50, max_width=50, p=1),
        ], p=0.3),
        # Geometric
        A.OneOf([
            A.Rotate(limit=15, p=1),
            A.Perspective(scale=(0.05, 0.1), p=1),
        ], p=0.3),
        # Color shifts
        A.OneOf([
            A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=1),
        ], p=0.3),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

def simulate_night_vision(image):
    """Create night vision effect"""
    dark = cv2.convertScaleAbs(image, alpha=0.3, beta=-50)
    gray = cv2.cvtColor(dark, cv2.COLOR_BGR2GRAY)
    night = cv2.applyColorMap(gray, cv2.COLORMAP_BONE)
    noise = np.random.normal(0, 15, night.shape).astype(np.uint8)
    return cv2.add(night, noise)

def add_lens_occlusion(image, severity='medium'):
    """Simulate dust on lens"""
    overlay = image.copy()
    h, w = image.shape[:2]
    num_spots = {'light': 2, 'medium': 4, 'heavy': 7}[severity]
    
    for _ in range(num_spots):
        x = np.random.randint(0, w)
        y = np.random.randint(0, h)
        radius = np.random.randint(30, 80)
        cv2.circle(overlay, (x, y), radius, (180, 160, 140), -1)
    
    return cv2.addWeighted(image, 0.6, overlay, 0.4, 0)

print("✅ Augmentation functions defined!")

✅ Augmentation functions defined!


In [7]:
def augment_dataset(input_dir, output_dir, num_augmentations=3):
    """
    Augment dataset
    Args:
        input_dir: Path to original data (should contain 'images' and 'labels' folders)
        output_dir: Where to save augmented data
        num_augmentations: How many augmented versions per image
    """
    transform = get_augmentation_pipeline()
    
    images_in = Path(input_dir) / 'images'
    labels_in = Path(input_dir) / 'labels'
    images_out = Path(output_dir) / 'images'
    labels_out = Path(output_dir) / 'labels'
    
    images_out.mkdir(parents=True, exist_ok=True)
    labels_out.mkdir(parents=True, exist_ok=True)
    
    image_files = list(images_in.glob('*.*'))
    print(f"Found {len(image_files)} images to augment...")
    
    for img_path in tqdm(image_files, desc="Augmenting"):
        # Load image
        image = cv2.imread(str(img_path))
        if image is None:
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load labels
        label_path = labels_in / f"{img_path.stem}.txt"
        bboxes = []
        class_labels = []
        
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    class_labels.append(int(parts[0]))
                    bboxes.append(list(map(float, parts[1:5])))
        
        # Copy original
        shutil.copy(img_path, images_out / img_path.name)
        if label_path.exists():
            shutil.copy(label_path, labels_out / label_path.name)
        
        # Create augmentations
        for aug_idx in range(num_augmentations):
            try:
                if aug_idx < num_augmentations - 1:
                    # Standard augmentation
                    augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                    aug_image = augmented['image']
                    aug_bboxes = augmented['bboxes']
                    aug_labels = augmented['class_labels']
                elif aug_idx == num_augmentations - 1 and np.random.random() > 0.5:
                    # Night vision
                    aug_image = simulate_night_vision(image)
                    aug_bboxes = bboxes
                    aug_labels = class_labels
                else:
                    # Lens occlusion
                    aug_image = add_lens_occlusion(image)
                    aug_bboxes = bboxes
                    aug_labels = class_labels
                
                # Save image
                aug_name = f"{img_path.stem}_aug{aug_idx}{img_path.suffix}"
                aug_image_bgr = cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR)
                cv2.imwrite(str(images_out / aug_name), aug_image_bgr)
                
                # Save labels
                label_out = labels_out / f"{img_path.stem}_aug{aug_idx}.txt"
                with open(label_out, 'w') as f:
                    for cls_id, bbox in zip(aug_labels, aug_bboxes):
                        f.write(f"{cls_id} {' '.join(map(str, bbox))}\n")
            except Exception as e:
                print(f"Error on {img_path.name}: {e}")

print("✅ Augmentation pipeline ready!")

✅ Augmentation pipeline ready!


In [2]:
!ls /kaggle/input/autonomous-drone-landing-dataset-teknofest


data.yaml  train  valid


In [8]:
INPUT_TRAIN = '/kaggle/input/autonomous-drone-landing-dataset-teknofest/train/train'
INPUT_VALID = '/kaggle/input/autonomous-drone-landing-dataset-teknofest/valid/valid'

In [9]:
OUTPUT_TRAIN = '/kaggle/working/augmented/train'
OUTPUT_VALID = '/kaggle/working/augmented/valid'

In [10]:
print("Checking paths...")
print(f"Train exists: {Path(INPUT_TRAIN).exists()}")
print(f"Valid exists: {Path(INPUT_VALID).exists()}")

Checking paths...
Train exists: True
Valid exists: True


In [11]:
def is_valid_yolo_bbox(b):
    _, x, y, w, h = b
    return 0 < w <= 1 and 0 < h <= 1 and 0 <= x <= 1 and 0 <= y <= 1


In [12]:
print("="*60)
print("STARTING DATA AUGMENTATION")
print("="*60)
print("\nThis will take 10-30 minutes...\n")

# Augment training data
print("1. Augmenting training set (3 versions per image)...")
augment_dataset(INPUT_TRAIN, OUTPUT_TRAIN, num_augmentations=3)

# Augment validation data
print("\n2. Augmenting validation set (2 versions per image)...")
augment_dataset(INPUT_VALID, OUTPUT_VALID, num_augmentations=2)

# Show results
original_train = len(list(Path(f"{INPUT_TRAIN}/images").glob('*.*')))
augmented_train = len(list(Path(f"{OUTPUT_TRAIN}/images").glob('*.*')))
original_valid = len(list(Path(f"{INPUT_VALID}/images").glob('*.*')))
augmented_valid = len(list(Path(f"{OUTPUT_VALID}/images").glob('*.*')))

print("\n" + "="*60)
print("AUGMENTATION COMPLETE!")
print("="*60)
print(f"Training:   {original_train} → {augmented_train} ({augmented_train/original_train:.1f}x)")
print(f"Validation: {original_valid} → {augmented_valid} ({augmented_valid/original_valid:.1f}x)")

/tmp/ipykernel_55/2157672548.py:12: UserWarning: Argument(s) 'fog_coef_lower, fog_coef_upper' are not valid for transform RandomFog
  A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3, p=1),
/tmp/ipykernel_55/2157672548.py:13: UserWarning: Argument(s) 'slant_lower, slant_upper' are not valid for transform RandomRain
  A.RandomRain(slant_lower=-10, slant_upper=10, p=1),
/tmp/ipykernel_55/2157672548.py:24: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10, 50), p=1),
/tmp/ipykernel_55/2157672548.py:26: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=5, max_height=50, max_width=50, p=1),


STARTING DATA AUGMENTATION

This will take 10-30 minutes...

1. Augmenting training set (3 versions per image)...
Found 2880 images to augment...


Augmenting:   1%|          | 15/2880 [00:08<25:41,  1.86it/s] 

Error on frame_008792.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].
Error on frame_008792.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].


Augmenting:   2%|▏         | 69/2880 [00:37<28:14,  1.66it/s]

...

Error on frame_008604.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].


Augmenting:  96%|█████████▋| 694/720 [04:05<00:06,  3.80it/s]

Error on frame_008720.jpg: y_max is less than or equal to y_min for bbox [   0.068359           0    0.079568           0           3].


Augmenting: 100%|██████████| 720/720 [04:13<00:00,  2.84it/s]


AUGMENTATION COMPLETE!
Training:   2880 → 11178 (3.9x)
Validation: 720 → 2104 (2.9x)


In [16]:
data_config = {
    'path': '/kaggle/working/augmented',
    'train': 'train/images',
    'val': 'valid/images',
    'nc': 4,
    'names': ['Vehicle', 'UAP', 'UAI', 'Person']
}

yaml_path = '/kaggle/working/augmented/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

In [13]:
!pip install -q ultralytics

In [14]:
!mkdir -p /kaggle/working/results/metrics

In [17]:
from ultralytics import YOLO
import torch
import numpy as np
import random
""
DATA_YAML = "/kaggle/working/augmented/data.yaml"

def train_ensemble_member(seed: int, name: str, i: int):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    model = YOLO("yolo11n.pt")

    model.train(
        data=DATA_YAML,
        epochs=60,
        imgsz=640,
        batch=16,
        name=name,
        patience=12,
        device=0,
        hsv_h=min(0.02, 0.015 + i * 0.002),
        hsv_s=min(1.0, 0.7 + i * 0.05),
        degrees=10 + i * 2,
        lr0=0.01 * (1 + i * 0.1),
        exist_ok=True
    )

    metrics = model.val()
    return {
        "map50": float(metrics.box.map50),
        "map": float(metrics.box.map)
    }
ensemble_metrics = {}

for i in range(5):
    print("=" * 50)
    print(f"Training ensemble member {i+1}/5")
    print("=" * 50)

    metrics = train_ensemble_member(
        seed=42 + i * 100,
        name=f"ensemble_member_{i+1}",
        i=i
    )

    ensemble_metrics[f"member_{i+1}"] = metrics
    print(f"Member {i+1} mAP@0.5: {metrics['map50']:.3f}")


with open("/kaggle/working/results/metrics/ensemble_metrics.json", "w") as f:
    json.dump(ensemble_metrics, f, indent=2)

print("✅ Ensemble training complete")


Training ensemble member 1/5
Ultralytics 8.4.12 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/augmented/data.yaml, degrees=10, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=ensemble_member_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

KeyboardInterrupt: 

In [21]:
!zip -r ensemble_member_3.zip runs/detect/ensemble_member_3


  adding: runs/detect/ensemble_member_3/ (stored 0%)
  adding: runs/detect/ensemble_member_3/train_batch0.jpg (deflated 5%)
  adding: runs/detect/ensemble_member_3/val_batch1_labels.jpg (deflated 7%)
  adding: runs/detect/ensemble_member_3/val_batch2_pred.jpg (deflated 5%)
  adding: runs/detect/ensemble_member_3/confusion_matrix.png (deflated 26%)
  adding: runs/detect/ensemble_member_3/val_batch0_labels.jpg (deflated 7%)
  adding: runs/detect/ensemble_member_3/train_batch2.jpg (deflated 3%)
  adding: runs/detect/ensemble_member_3/val_batch0_pred.jpg (deflated 7%)
  adding: runs/detect/ensemble_member_3/weights/ (stored 0%)
  adding: runs/detect/ensemble_member_3/weights/last.pt (deflated 10%)
  adding: runs/detect/ensemble_member_3/weights/best.pt (deflated 10%)
  adding: runs/detect/ensemble_member_3/BoxR_curve.png (deflated 11%)
  adding: runs/detect/ensemble_member_3/train_batch13982.jpg (deflated 9%)
  adding: runs/detect/ensemble_member_3/val_batch1_pred.jpg (deflated 7%)
  addin

In [19]:
from ultralytics import YOLO
import torch
import numpy as np
import random
import json

DATA_YAML = "/kaggle/working/augmented/data.yaml"

def train_ensemble_member(seed: int, name: str, i: int):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    model = YOLO("yolo11n.pt")

    model.train(
        data=DATA_YAML,
        epochs=30,              # ✅ ONLY 30 epochs
        imgsz=640,
        batch=16,
        name=name,
        patience=12,
        device=0,
        hsv_h=min(0.02, 0.015 + i * 0.002),
        hsv_s=min(1.0, 0.7 + i * 0.05),
        degrees=10 + i * 2,
        lr0=0.01 * (1 + i * 0.1),
        exist_ok=True
    )

    metrics = model.val()
    return {
        "map50": float(metrics.box.map50),
        "map": float(metrics.box.map)
    }

ensemble_metrics = {}

# 🔴 START FROM MEMBER 2 (i = 1)
for i in range(1, 5):
    print("=" * 50)
    print(f"Training ensemble member {i+1}/5 (30 epochs)")
    print("=" * 50)

    metrics = train_ensemble_member(
        seed=42 + i * 100,
        name=f"ensemble_member_{i+1}",
        i=i
    )

    ensemble_metrics[f"member_{i+1}"] = metrics
    print(f"Member {i+1} mAP@0.5: {metrics['map50']:.3f}")

with open("/kaggle/working/results/metrics/ensemble_metrics_2to5.json", "w") as f:
    json.dump(ensemble_metrics, f, indent=2)

print("✅ Ensemble members 2–5 training complete")


Training ensemble member 2/5 (30 epochs)
Ultralytics 8.4.12 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/augmented/data.yaml, degrees=12, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.017, hsv_s=0.75, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.011000000000000001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=ensemble_member_2, nbs=64, nms=False, opset=None, optimi

Exception in thread Thread-94 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd


KeyboardInterrupt: 

    fd = df.detach()
         ^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/resource_sharer.py", line 86, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 519, in Client
    c = SocketClient(address)
        ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 647, in SocketClient
    s.connect(address)
FileNotFoundError: [Errno 2] No such file or directory


In [22]:
!ls runs/detect


ensemble_member_1  ensemble_member_3  val   val3
ensemble_member_2  ensemble_member_4  val2


In [23]:
import numpy as np
import cv2
from ultralytics import YOLO
from typing import List


In [24]:
class EnsembleYOLO:
    """
    Deep ensemble for uncertainty quantification
    Adapted for 3 trained models (hackathon-safe)
    """
    def __init__(self, model_paths: List[str]):
        """
        Args:
            model_paths: List of paths to trained ensemble member weights
        """
        self.models = []

        for path in model_paths:
            try:
                model = YOLO(path)
                self.models.append(model)
                print(f"✅ Loaded model: {path}")
            except Exception as e:
                print(f"⚠️ Failed to load {path}: {e}")

        self.n_models = len(self.models)

        if self.n_models == 0:
            raise RuntimeError("❌ No ensemble models loaded")

        print(f"\n🎯 Ensemble initialized with {self.n_models} models")

    def predict_with_uncertainty(self, image_path: str) -> dict:
        """
        Run inference using all ensemble members
        """
        all_predictions = []

        for model in self.models:
            results = model(image_path, verbose=False)
            all_predictions.append(results[0])

        return self._aggregate_ensemble(all_predictions, image_path)

    def _aggregate_ensemble(self, predictions: List, image_path: str) -> dict:
        """
        Aggregate ensemble predictions to compute consensus & disagreement
        """
        img = cv2.imread(image_path)
        h, w = img.shape[:2]

        pad_detections = []
        confidences = []

        # Collect detections from all models
        for pred in predictions:
            boxes = pred.boxes
            if boxes is not None:
                for box in boxes:
                    class_id = int(box.cls[0])

                    # UAP / UAI only
                    if class_id in [1, 2]:
                        conf = float(box.conf[0])
                        bbox = box.xyxy[0].cpu().numpy()

                        pad_detections.append({
                            "class_id": class_id,
                            "confidence": conf,
                            "bbox": bbox
                        })
                        confidences.append(conf)

        # Consensus: how many models detected a pad
        models_with_detection = 0
        for pred in predictions:
            if pred.boxes is not None:
                if any(int(box.cls[0]) in [1, 2] for box in pred.boxes):
                    models_with_detection += 1

        consensus = models_with_detection / self.n_models

        # Disagreement (normalized std for small ensembles)
        if len(confidences) > 1 and np.mean(confidences) > 0:
            disagreement = np.std(confidences) / np.mean(confidences)
        else:
            disagreement = 1.0

        best_pad = (
            max(pad_detections, key=lambda x: x["confidence"])
            if pad_detections else None
        )

        return {
            "best_pad_detection": best_pad,
            "consensus": consensus,
            "disagreement": disagreement,
            "mean_confidence": float(np.mean(confidences)) if confidences else 0.0,
            "num_models_agree": models_with_detection,
            "total_models": self.n_models
        }


In [25]:
ensemble = EnsembleYOLO([
    "runs/detect/ensemble_member_1/weights/best.pt",
    "runs/detect/ensemble_member_2/weights/best.pt",
    "runs/detect/ensemble_member_3/weights/best.pt",
])


✅ Loaded model: runs/detect/ensemble_member_1/weights/best.pt
✅ Loaded model: runs/detect/ensemble_member_2/weights/best.pt
✅ Loaded model: runs/detect/ensemble_member_3/weights/best.pt

🎯 Ensemble initialized with 3 models


In [29]:
!ls /kaggle/input/autonomous-drone-landing-dataset-teknofest/valid/valid/images/frame_000000

ls: cannot access '/kaggle/input/autonomous-drone-landing-dataset-teknofest/valid/valid/images/frame_000000': No such file or directory


In [32]:
test_image = "/kaggle/input/autonomous-drone-landing-dataset-teknofest/valid/valid/images/frame_000016.jpg"  # adjust if needed

result = ensemble.predict_with_uncertainty(test_image)

print("====== ENSEMBLE RESULT ======")
print(f"Consensus: {result['consensus']:.2%}")
print(f"Disagreement: {result['disagreement']:.3f}")
print(f"Models in agreement: {result['num_models_agree']}/{result['total_models']}")

if result["best_pad_detection"]:
    print("\nBest pad detection:")
    print(f"  Class ID: {result['best_pad_detection']['class_id']}")
    print(f"  Confidence: {result['best_pad_detection']['confidence']:.3f}")
else:
    print("\nNo landing pad detected")


====== ENSEMBLE RESULT ======
Consensus: 100.00%
Disagreement: 0.012
Models in agreement: 3/3

Best pad detection:
  Class ID: 1
  Confidence: 0.967
